In [2]:
import requests
import json
import time
from datetime import datetime,timedelta
from kafka import KafkaProducer

In [5]:
def get_news():
    url = 'https://api.currentsapi.services/v1/latest-news'
    params = {
    'apiKey': 'DsvYr_X-YCRPVNbLtAkCiF4oCREpHl7BlRjjo5Q9J66RRQPj',
    'country': 'us',
    'language': 'en',
    'category':'technology,business',
    'page': 1,
    'published_after': (datetime.utcnow() - timedelta(hours=1)).isoformat(),
    'source': 'bbc,cnn'}

    try:
        r = requests.get(url, params=params)
        if r.status_code == 200:
            data = r.json()
            articles = data["news"]
            for article in articles:
                yield json.dumps(article)
        else:
            for i in range(3):
                time.sleep(3)
                r = requests.get(url, params=params)
                if r.status_code == 200:
                    data = r.json()
                    articles = data["news"]
                    for article in articles:
                        yield json.dumps(article)
                    break
                else:
                    print(f"There was a problem with the connection. Trial number {i+1}, Status code: {r.status_code}")
    except requests.exceptions.RequestException as e:
        print("Request Error",e)
    except json.JSONDecodeError as e:
        print("Json Decode Error: ",e)
    except Exception as e:
        print("Error:", e)



def stream_data():
    producer = KafkaProducer(bootstrap_servers=["ed-kafka:29092"])
    current_time = time.time()
    
    while True:
        if time.time() > current_time + 60:
            break
        
        try:
            for article in get_news():
                news_ = json.loads(article)
                category = news_.get("category", [])
                
                if isinstance(category, list) and "technology" in category:
                    producer.send("technology", json.dumps(news_).encode("utf-8"))
                
                if isinstance(category,list) and "business" in category:
                    producer.send("business",json.dumps(news_).encode("utf-8"))
        
        except json.JSONDecodeError as je:
            logging.error(f'JSON decoding error: {je}')
        
        except KafkaError as ke:
            logging.error(f'Error while sending to Kafka: {ke}')
        
        except Exception as e:
            logging.error(f'An error occurred: {e}')
            continue
    
    producer.flush()
    producer.close()
stream_data()